In [11]:
import requests
import cv2
import os
import numpy as np

In [6]:


# Getting Image and store it
url = 'http://10.12.105.245:8080/shot.jpg'
def get_image(url):
    img_resp = requests.get(url)
    img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
    img = cv2.imdecode(img_arr, -1)


In [12]:
def get_image(url, save_path):
    for i in range (100):
        img_resp = requests.get(url)
        img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
        img = cv2.imdecode(img_arr, -1)
        
        # Check if the save path exists, create it if not
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        
        # Save the image with a unique name in the specified folder
        img_name = os.path.join(save_path, f'image{i}.jpg')
        cv2.imwrite(img_name, img)
    
    return img_name

In [13]:
save_folder = 'images'
saved_image_path = get_image(url, save_folder)

In [27]:
howmanypics = 10
personName = "Raj"

In [29]:
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def face_cropped(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.2, 5)
    if len(faces) != 1:
        return None
    x, y, w, h = faces[0]
    cropped_face = img[y: y+h, x:x+w]
    return cropped_face

def process_images(url_template, save_path, personName):
    for i in range(1, howmanypics+1):  # Process 100 images
        url = url_template.format(i)
        img_resp = requests.get(url)
        img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
        img = cv2.imdecode(img_arr, -1)

        # Check if a face is detected in the first image
        if i == 1:
            face = face_cropped(img)
            if face is None:
                print("No face detected in the first image. Exiting.")
                return
        
        # Process and save the face as a 64x64 image
        face = face_cropped(img)
        face_resized = cv2.resize(face, (128, 128))
        face_gray = cv2.cvtColor(face_resized, cv2.COLOR_BGR2GRAY)
        file_path = os.path.join(save_path, f'{personName}_{i}.jpg')
        cv2.imwrite(file_path, face_gray)

    print("Faces cropped and saved successfully.")

# Example usage # URL template for 100 images
save_path = 'images'  # Specify where to save the cropped faces
process_images(url, save_path, personName)

Faces cropped and saved successfully.


In [22]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical

In [30]:
def load_images(directory, image_size):
    images = []
    labels = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg"):
            img = cv2.imread(os.path.join(directory, filename))
            img = cv2.resize(img, image_size)
            images.append(img)
            labels.append(filename.split('_')[0])  # Assuming filename format: "PersonName_index.jpg"
    return np.array(images), np.array(labels)

# Set your dataset directory and image size
dataset_dir = 'images'
image_size = (128, 128)

# Load images and labels
images, labels = load_images(dataset_dir, image_size)

# Encode labels as integers
label_mapping = {label: idx for idx, label in enumerate(np.unique(labels))}
encoded_labels = np.array([label_mapping[label] for label in labels])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, encoded_labels, test_size=0.3, random_state=42)

# Convert labels to one-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [31]:

# Define the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(len(label_mapping), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy}")





Epoch 1/10


1/1 [==============================] - 2s 2s/step - loss: 20.2244 - accuracy: 0.5714 - val_loss: 531.3474 - val_accuracy: 0.6667
Epoch 2/10
1/1 [==============================] - 0s 187ms/step - loss: 931.0374 - accuracy: 0.4286 - val_loss: 255.3411 - val_accuracy: 0.6667
Epoch 3/10
1/1 [==============================] - 0s 159ms/step - loss: 449.8055 - accuracy: 0.4286 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 159ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 83.6134 - val_accuracy: 0.3333
Epoch 5/10
1/1 [==============================] - 0s 158ms/step - loss: 41.7930 - accuracy: 0.5714 - val_loss: 36.1216 - val_accuracy: 0.3333
Epoch 6/10
1/1 [==============================] - 0s 152ms/step - loss: 8.6859 - accuracy: 0.5714 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 158ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_acc

In [40]:
# Example usage for prediction
test_image = cv2.imread('images/test.jpg')
test_image = cv2.resize(test_image, image_size)
test_image = np.expand_dims(test_image, axis=0)
prediction = model.predict(test_image)
predicted_label = list(label_mapping.keys())[np.argmax(prediction)]
print(f"Predicted Label: {predicted_label}")

1/1 [==============================] - 0s 196ms/step
Predicted Label: Aditya


In [ ]:
def face_cropped_from_list(images_list):
    # Load the face classifier
    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    all_faces_list = []

    for img in images_list:
        faces_list = []
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Detect faces in the image
        faces = face_classifier.detectMultiScale(gray, 1.2, 5)

        # Iterate through detected faces
        for (x, y, w, h) in faces:
            # Crop the face from the original image
            cropped_face = img[y: y+h, x:x+w]

            # Resize the face to a common size (e.g., 200x200)
            cropped_face = cv2.resize(cropped_face, (200, 200))

            # Convert the face to grayscale
            cropped_face = cv2.cvtColor(cropped_face, cv2.COLOR_BGR2GRAY)

            # Append the face to the faces_list
            faces_list.append(cropped_face)

        # Append the faces_list to the all_faces_list
        all_faces_list.append(faces_list)

    return all_faces_list

# all_faces_list = face_cropped_from_list(captured_images)

def recognize_faces(image_lists, model_path='RecognitionModel.h5', map_path='StudentsMap.pkl'):
    # Load the model
    loaded_model = load_model(model_path)

    # Load the name mapping
    with open(map_path, 'rb') as file:
        loaded_object = pickle.load(file)

    # Initialize the result list
    results = []

    # Process each list of images
    for image_list in image_lists:
        # Initialize the list for this set of images
        names = []

        # Process each image in the list
        for img_array in image_list:
            resized_image = cv2.resize(img_array, (64, 64))
            gray_image = resized_image

            # Expand dimensions to make it compatible with the model input shape
            test_image = np.expand_dims(gray_image, axis=0)
            test_image = np.expand_dims(test_image, axis=-1)  # Add channel dimension for grayscale

            # Now, you can use the loaded model for prediction
            result = loaded_model.predict(test_image, verbose=0)
            names.append(loaded_object[np.argmax(result)])


        # Append the list of names for this set of images to the result
        results.append(names)

    return results

In [39]:

def capture_test(url_template, save_path):
    img_resp = requests.get(url)
    img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
    img = cv2.imdecode(img_arr, -1)

    
    face = face_cropped(img)
    if face is None:
        print("No face detected in the first image. Exiting.")
        return
    
    # Process and save the face as a 64x64 image
    face = face_cropped(img)
    face_resized = cv2.resize(face, (128, 128))
    face_gray = cv2.cvtColor(face_resized, cv2.COLOR_BGR2GRAY)
    file_path = os.path.join(save_path, f'test.jpg')
    cv2.imwrite(file_path, face_gray)

    print("Faces cropped and saved successfully.")

# Example usage # URL template for 100 images
save_path = 'images'  # Specify where to save the cropped faces
capture_test(url, save_path)

Faces cropped and saved successfully.
